In [1]:
!mkdir predicted_frames
!mkdir save_frames

#Run the snippets above first

mkdir: cannot create directory ‘predicted_frames’: File exists
mkdir: cannot create directory ‘save_frames’: File exists


In [2]:
# importing the necessary libraries
import cv2
import numpy as np
import uuid
import os

# Creating a VideoCapture object to read the video
cap = cv2.VideoCapture('data/test_video.mp4')

count=0
while (cap.isOpened()):

    ret, frame = cap.read()
    
    filename=str(uuid.uuid4())+".jpg"
    fullpath=os.path.join("./save_frames",filename)
    cv2.imwrite(fullpath, frame)
    count+=1
    
    if count==20:
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
!ls

data	   predicted_frames  save_frames	       video_processing.ipynb
image.png  README.md	     Solution_Methodology.pdf
lib	   requirements.txt  src


In [4]:
import tensorflow as tf
print(tf.__version__)

2024-11-22 11:57:08.462704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732247828.487138  112689 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732247828.494806  112689 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 11:57:08.519538: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0


In [5]:
predicted_path = './predicted_frames'

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import zipfile
import cv2
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import uuid
import time

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

KPIs_dict = dict()

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = "data/frozen_inference_graph.pb"

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = "data/labels.txt"

NUM_CLASSES = 7

# ## Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# ## Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# ## Helper code
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def calculate_area(x1, y1, x2, y2):
    xDiff = abs(x1 - x2)
    yDiff = abs(y1 - y2)
    area = xDiff * yDiff
    return area

# Size, in inches, of the output images.
IMAGE_SIZE = (500, 500)
count = 0
frame_number = 0
cap = cv2.VideoCapture('data/test_video.mp4')

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while cap.isOpened():
            frame_number += 1

            ret, frame = cap.read()

            filename = str(uuid.uuid4()) + ".jpg"
            fullpath = os.path.join("./save_frames", filename)
            cv2.imwrite(fullpath, frame)
            count += 1

            # for testing script...
            if count == 50:
                break
            # ends here

            image = Image.open(fullpath)

            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.

            result = vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)

            # Unpack the result based on the actual number of returned values
            if isinstance(result, tuple) and len(result) == 2:
                image, box_to_display_str_map = result
            elif isinstance(result, tuple) and len(result) == 3:
                image, box_to_display_str_map, additional_value = result
            else:
                raise ValueError("Unexpected number of values returned from visualize_boxes_and_labels_on_image_array")

            image_pil = Image.fromarray(np.uint8(image_np)).convert('RGB')
            im_width, im_height = image_pil.size
            area_whole = im_width * im_height
            for key, value in box_to_display_str_map.items():
                ymin, xmin, ymax, xmax = key
                (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
                area = calculate_area(top, left, bottom, right)

                percent_area = round(area / area_whole, 2)
                rindex = value[0].rfind(':')
                brand_name = value[0][:rindex]

                if brand_name in KPIs_dict.keys():
                    KPIs_dict[brand_name]['count'] += 1
                    KPIs_dict[brand_name]['area'].append(percent_area)
                    KPIs_dict[brand_name]['frames'].append(frame_number)
                else:
                    KPIs_dict[brand_name] = {"count": 1}
                    KPIs_dict[brand_name].update({"area": [percent_area]})
                    KPIs_dict[brand_name].update({"frames": [frame_number]})

            full_predicted_path = os.path.join(predicted_path, str(uuid.uuid4()) + ".jpg")
            cv2.imwrite(full_predicted_path, image)

I0000 00:00:1732248042.924146  112689 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6887 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1


ValueError: Unexpected number of values returned from visualize_boxes_and_labels_on_image_array

In [ ]:
KPIs_dict

In [ ]:
for each_brand,analytics_dict in KPIs_dict.items():
    area=analytics_dict['area']
    response = shortest_longest_area(area)
    KPIs_dict[each_brand].update(response)

In [ ]:
def shortest_longest_area(area_list):
    area_list.sort()
    shortest = area_list[0]
    longest = area_list[-1]
    response = {
        "shortest":shortest,
        "longest":longest
    }
    return response

In [ ]:
KPIs_dict